# adaboost final

## Import Library

In [1]:
from sklearn import datasets
from sklearn. tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
import numpy as np
from sklearn.model_selection import train_test_split
from math import e
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## adaboost

In [2]:
class adaboost:
  def __init__(self, num_of_model = 20,lr=0.1 ):
    self.num_of_model = num_of_model
    self.lr = lr
    self.clfs = []
    self.clfs_weight = []
    self.clfs_weight_final = []
    self.rj_final = []
    self.alpha_final = []
    self.update_weight_final = []
    self.y_pred_final_fit = []
    self.y_pred_every_clf=[]
    self.final_result = []

  def CalcWeightedError(self,y_true, y_pred):
    # kalo nilainya tidak sama +1
    return np.sum(y_true != y_pred) / len(y_pred)

  def CalcError(self,y_true, y_pred):
    # menghasilkan True apabila y_true (target aktual) tidak sesuai dengan nilai aktual
    res = np.array([y_true != y_pred])
    res = res.astype(int)
    return res

  def UpdateWeights(self,weights, pred_weight, error):
    new_weights = weights * (e ** (pred_weight * error))
    new_weights = new_weights/np.sum(new_weights)
    return new_weights

  def CalcPredWeight(self,lr, weighted_error):
    return self.lr * np.log((1 - weighted_error + 1)/(weighted_error + 1))

  def fit(self,X,y):
    X_train = X
    m, n = X_train.shape
    y_train = y
    weights = np.full(len(X_train), 1/len(X_train))
    for i in range(self.num_of_model):
      # print(i)
      # inisialisasi classifier
      clf = KNeighborsClassifier(n_neighbors=3)
      # fitting data
      clf.fit(X_train, y_train)
      # append clf to clfs
      self.clfs.append(clf)
      # predict X_train
      y_pred = clf.predict(X_train)
      # append y_pred to y_pred_final_fit
      self.y_pred_final_fit.append(y_pred)
      # print("ini y pred fit",y_pred)

      # menghitung persamaan 2 (rj)
      weighted_error = self.CalcWeightedError(y_train, y_pred)
      # menghitung persamaan 3 (alpha)
      pred_weight = self.CalcPredWeight(self.lr, weighted_error)
      # menambahkan persamaan 3 to clfs_weight
      self.clfs_weight.append(pred_weight)

      # print("ini self.clfs_weight",self.clfs_weight)
      # perhitungan error
      error = self.CalcError(y_train, y_pred)
      # bobot baru
      new_weights = self.UpdateWeights(weights, pred_weight, error)
      # print("ini new_weigth",new_weights)

      # index baru
      new_indices = np.random.choice(m, m, p=new_weights.flatten())
      # print("new_indices",new_indices)
      X_train = X_train[new_indices]
      # print(X_train)
      y_train = y_train[new_indices]



  def predict(self,x,y):
    n_estimator = len(self.clfs)
    m=len(x)
    # membuat matrix m x n_estimator
    constructor=[[str(i)+str(j) for i in range(n_estimator)] for j in range(m)]
    # mengisi konstruksi dengan hasil prediktor
    for i in range(n_estimator):
      clf = self.clfs[i]
      for u in range(m):
        constructor[u][i]=clf.predict([x[u]])
    self.y_pred_every_clf=constructor
    # mengisi prediksi akhir
    # perulangan sebanyak data
    for i in range(len(self.y_pred_every_clf)):
      y_pred_temp=[]
      weight_temp=[]
      # perulangan clf sebanyak n_estimator(banyaknya model)
      for u in range(len(self.y_pred_every_clf[0])):
        if self.y_pred_every_clf[i][u] not in y_pred_temp:
          y_pred_temp.append(self.y_pred_every_clf[i][u])
          weight_temp.append(self.clfs_weight[u])
        else:
          for o in range(len(y_pred_temp)):
            if self.y_pred_every_clf[i][u]==y_pred_temp[o]:
              weight_temp[o]+=self.clfs_weight[u]
      self.clfs_weight_final.append(weight_temp)
      indexnya=weight_temp.index(max(weight_temp))
      self.final_result.append(y_pred_temp[indexnya])
    return self.final_result



In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/aliefakbrp/dataset/main/lung_cancer_patient_datasets.csv')
dataset=dataset.drop(columns=['index', 'Patient Id'])
X = dataset.drop(columns=["Level"])
y = dataset["Level"]

In [4]:
X

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Coughing of Blood,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring
0,33,1,2,4,5,4,3,2,2,4,...,4,3,4,2,2,3,1,2,3,4
1,17,1,3,1,5,3,4,2,2,2,...,3,1,3,7,8,6,2,1,7,2
2,35,1,4,5,6,5,5,4,6,7,...,8,8,7,9,2,1,4,6,7,2
3,37,1,7,7,7,7,6,7,7,7,...,8,4,2,3,1,4,5,6,7,5
4,46,1,6,8,7,7,7,6,7,7,...,9,3,2,4,1,4,2,4,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,44,1,6,7,7,7,7,6,7,7,...,7,5,3,2,7,8,2,4,5,3
996,37,2,6,8,7,7,7,6,7,7,...,7,9,6,5,7,2,4,3,1,4
997,25,2,4,5,6,5,5,4,6,7,...,8,8,7,9,2,1,4,6,7,2
998,18,2,6,8,7,7,7,6,7,7,...,9,3,2,4,1,4,2,4,2,3


In [5]:
X=X.values.tolist()
X = np.array(X)

In [6]:
y = np.array(y)

## Data Test

In [7]:

from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5)
clf = adaboost()
clf.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
clf.predict(X_test,y_test)
print(y_test[1])
clf.final_result[1]

Medium


array(['Medium'], dtype=object)

## Inputan User

In [8]:
Age = int(input("Age"))
Gender = int(input("Gender"))
Air_Pollution = int(input("Air Pollution"))
Alcohol_Use = int(input("Alcohol use"))
Dust_Allergy = int(input("Dust Allergy"))
OccuPational_Hazards = int(input("OccuPational Hazards"))
Genetic_Risk = int(input("Genetic Risk"))
chronic_Lung_Disease = int(input("chronic Lung Disease"))
Balanced_Diet = int(input("Balanced Diet"))
Obesity = int(input("Obesity"))
Smoking = int(input("Smoking"))
Passive_Smoker = int(input("Passive Smoker"))
Chest_Pain = int(input("Chest Pain"))
Coughing_of_Blood = int(input("Coughing of Blood"))
Fatigue = int(input("Fatigue"))
Weight_Loss = int(input("Weight Loss"))
Shortness_of_Breath = int(input("Shortness of Breath"))
Wheezing = int(input("Wheezing"))
Swallowing_Difficulty = int(input("Swallowing Difficulty"))
Clubbing_of_Finger_Nails = int(input("Clubbing of Finger Nails"))
Frequent_Cold = int(input("Frequent Cold"))
Dry_Cough = int(input("Dry Cough"))
Snoring = int(input("Snoring"))

Age33
Gender1
Air Pollution2
Alcohol use2
Dust Allergy3
OccuPational Hazards3
Genetic Risk3
chronic Lung Disease2
Balanced Diet2
Obesity2
Smoking2
Passive Smoker2
Chest Pain2
Coughing of Blood2
Fatigue1
Weight Loss1
Shortness of Breath1
Wheezing1
Swallowing Difficulty1
Clubbing of Finger Nails1
Frequent Cold1
Dry Cough1
Snoring1


In [9]:
inputan=[[Age,
Gender,
Air_Pollution,
Alcohol_Use,
Dust_Allergy,
OccuPational_Hazards,
Genetic_Risk,
chronic_Lung_Disease,
Balanced_Diet,
Obesity,
Smoking,
Passive_Smoker,
Chest_Pain,
Coughing_of_Blood,
Fatigue,
Weight_Loss,
Shortness_of_Breath,
Wheezing,
Swallowing_Difficulty,
Clubbing_of_Finger_Nails,
Frequent_Cold,
Dry_Cough,
Snoring]]

In [13]:

from sklearn import datasets
from sklearn.model_selection import train_test_split

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5)
clf = adaboost()
clf.fit(X_train, y_train)
from sklearn.metrics import accuracy_score
clf.predict(inputan,y_test)
# print(y_test[1])
clf.final_result

[array(['Low'], dtype=object)]